# 🌐 PARTIE 7 : APIs ET SERVICES WEB

### 🌟 Ce que vous allez apprendre :
- **Théorie** : Protocoles SOAP vs REST vs GraphQL
- **Architecture** : RESTful design et bonnes pratiques
- **FastAPI** : Framework moderne Python pour APIs
- **Sécurité** : Authentification JWT et OAuth2
- **Documentation** : Swagger/OpenAPI automatique
- **Tests** : Testing d'APIs avec pytest

### 🛠️ Prérequis :
Exécutez d'abord la cellule système ci-dessous.

---

In [4]:
exec(open('helpers/api_helper.py').read())

🌐 API Helper v1.0.0 chargé avec succès!
💡 Tapez api_helper.help('section') pour obtenir de l'aide
📚 Sections disponibles: 7.1.1, 7.1.2, 7.2.1, 7.2.2, 7.3.1, 7.3.2, 7.4.1, 7.4.2


---

## 📚 Section 7.1 : Théorie des APIs

### 🎯 Objectif :
Comprendre les différents types d'APIs et leurs cas d'usage.

### 📝 Étape 7.1.1 : SOAP vs REST vs GraphQL

**Instructions :**
Analyser les différences entre les protocoles d'API.

In [ ]:
# 📝 ÉTAPE 7.1.1 : Comparaison des protocoles API
# Exécutez cette cellule pour voir la comparaison interactive

api_helper.compare_protocols()

In [5]:
# 💡 Aide pour l'étape 7.1.1
api_helper.help("7.1.1")

### 📝 Étape 7.1.2 : Principes REST

**Instructions :**
Comprendre les 6 contraintes de l'architecture REST.

In [ ]:
# 📝 ÉTAPE 7.1.2 : Architecture RESTful
# Analysez ces exemples d'endpoints REST :

# ✅ Bonnes pratiques REST :
good_endpoints = {
    "GET /api/v1/users": "Récupérer tous les utilisateurs",
    "GET /api/v1/users/123": "Récupérer l'utilisateur 123",
    "POST /api/v1/users": "Créer un nouvel utilisateur", 
    "PUT /api/v1/users/123": "Modifier complètement l'utilisateur 123",
    "PATCH /api/v1/users/123": "Modifier partiellement l'utilisateur 123",
    "DELETE /api/v1/users/123": "Supprimer l'utilisateur 123"
}

# ❌ Mauvaises pratiques :
bad_endpoints = {
    "GET /api/getUsers": "Verbe dans l'URL",
    "POST /api/users/delete/123": "Action dans l'URL avec mauvaise méthode",
    "GET /api/user123": "ID collé au nom de ressource"
}

# 👇 Analysez et proposez des améliorations :

In [6]:
# 💡 Aide pour l'étape 7.1.2
api_helper.help("7.1.2")

---

## 🚀 Section 7.2 : FastAPI Basics

### 🎯 Objectif :
Créer votre première API avec FastAPI.

### 📝 Étape 7.2.1 : Première API FastAPI

**Instructions :**
Créer une API simple avec plusieurs endpoints.

In [ ]:
# 📝 ÉTAPE 7.2.1 : API FastAPI de base
# Créez votre première API avec ces endpoints :

from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from typing import List, Optional
import uvicorn
from datetime import datetime

# Créer l'application FastAPI
app = FastAPI(
    title="Mon API d'exemple",
    description="Une API pour apprendre FastAPI",
    version="1.0.0"
)

# Modèle Pydantic pour validation
class User(BaseModel):
    id: Optional[int] = None
    name: str
    email: str
    age: int
    created_at: Optional[datetime] = None

# Base de données simulée
users_db = []
next_id = 1

# 👇 Implémentez ces endpoints :
# GET / - Page d'accueil
# GET /users - Lister tous les utilisateurs  
# GET /users/{user_id} - Obtenir un utilisateur
# POST /users - Créer un utilisateur
# PUT /users/{user_id} - Modifier un utilisateur
# DELETE /users/{user_id} - Supprimer un utilisateur

In [7]:
# 💡 Aide pour l'étape 7.2.1
api_helper.help("7.2.1")

### 📝 Étape 7.2.2 : Validation et modèles Pydantic

**Instructions :**
Ajouter la validation avancée avec Pydantic.

In [ ]:
# 📝 ÉTAPE 7.2.2 : Validation avancée
# Créez des modèles Pydantic plus sophistiqués :

from pydantic import BaseModel, EmailStr, validator, Field
from typing import List, Optional
from enum import Enum

class UserRole(str, Enum):
    admin = "admin"
    user = "user"
    moderator = "moderator"

class UserCreate(BaseModel):
    name: str = Field(..., min_length=2, max_length=50, description="Nom de l'utilisateur")
    email: EmailStr = Field(..., description="Email valide")
    age: int = Field(..., ge=13, le=120, description="Âge entre 13 et 120 ans")
    role: UserRole = Field(default=UserRole.user, description="Rôle de l'utilisateur")
    
    @validator('name')
    def name_must_contain_space_or_single_word(cls, v):
        if len(v.split()) == 0:
            raise ValueError('Le nom ne peut pas être vide')
        return v.title()

class UserResponse(BaseModel):
    id: int
    name: str
    email: str
    age: int
    role: UserRole
    created_at: datetime
    
    class Config:
        from_attributes = True

# 👇 Modifiez votre API pour utiliser ces modèles :

In [ ]:
# 💡 Aide pour l'étape 7.2.2
api_helper.help("7.2.2")

---

## 🔒 Section 7.3 : Sécurité et Authentification

### 🎯 Objectif :
Sécuriser votre API avec JWT et OAuth2.

### 📝 Étape 7.3.1 : Authentification JWT

**Instructions :**
Implémenter l'authentification par token JWT.

In [ ]:
# 📝 ÉTAPE 7.3.1 : Authentification JWT
# Implémentez l'authentification avec JWT :

from fastapi import Depends, HTTPException, status
from fastapi.security import HTTPBearer, HTTPAuthorizationCredentials
from passlib.context import CryptContext
from jose import JWTError, jwt
from datetime import datetime, timedelta
import os

# Configuration JWT
SECRET_KEY = "your-secret-key-change-in-production"
ALGORITHM = "HS256"
ACCESS_TOKEN_EXPIRE_MINUTES = 30

# Configuration du hashage de mot de passe
pwd_context = CryptContext(schemes=["bcrypt"], deprecated="auto")
security = HTTPBearer()

# Modèles pour l'authentification
class UserLogin(BaseModel):
    email: str
    password: str

class Token(BaseModel):
    access_token: str
    token_type: str

# Base de données utilisateurs (simulée)
users_auth_db = {
    "admin@example.com": {
        "id": 1,
        "email": "admin@example.com",
        "hashed_password": "$2b$12$EixZaYVK1fsbw1ZfbX3OXePaWxn96p36WQoeG6Lruj3vjPGga31lW",  # secret
        "role": "admin"
    }
}

# 👇 Implémentez ces fonctions :
# - verify_password()
# - get_password_hash()
# - create_access_token()
# - verify_token()
# - get_current_user()
# - Endpoints /login et /protected

In [ ]:
# 💡 Aide pour l'étape 7.3.1
api_helper.help("7.3.1")

### 📝 Étape 7.3.2 : Middleware et CORS

**Instructions :**
Ajouter des middlewares pour la sécurité et le CORS.

In [ ]:
# 📝 ÉTAPE 7.3.2 : Middleware et sécurité
# Ajoutez ces middlewares à votre API :

from fastapi.middleware.cors import CORSMiddleware
from fastapi.middleware.gzip import GZipMiddleware
from fastapi import Request
import time

# Middleware CORS
app.add_middleware(
    CORSMiddleware,
    allow_origins=["http://localhost:3000", "http://localhost:8080"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Middleware de compression
app.add_middleware(GZipMiddleware, minimum_size=1000)

# Middleware personnalisé pour logging
@app.middleware("http")
async def log_requests(request: Request, call_next):
    start_time = time.time()
    response = await call_next(request)
    process_time = time.time() - start_time
    
    print(f"{request.method} {request.url.path} - {response.status_code} - {process_time:.4f}s")
    response.headers["X-Process-Time"] = str(process_time)
    return response

# 👇 Testez votre API avec les middlewares :

In [8]:
# 💡 Aide pour l'étape 7.3.2
api_helper.help("7.3.2")

---

## 🧪 Section 7.4 : Tests et Déploiement

### 🎯 Objectif :
Tester et déployer votre API en production.

### 📝 Étape 7.4.1 : Tests automatisés

**Instructions :**
Créer des tests complets pour votre API.

In [ ]:
# 📝 ÉTAPE 7.4.1 : Tests d'API
# Créez des tests avec pytest et TestClient :

from fastapi.testclient import TestClient
import pytest

# Client de test
client = TestClient(app)

# Tests de base
def test_read_main():
    response = client.get("/")
    assert response.status_code == 200
    assert "API" in response.json()["message"]

def test_create_user():
    user_data = {
        "name": "Test User",
        "email": "test@example.com",
        "age": 25,
        "role": "user"
    }
    response = client.post("/users", json=user_data)
    assert response.status_code == 201
    assert response.json()["email"] == user_data["email"]

def test_get_users():
    response = client.get("/users")
    assert response.status_code == 200
    assert isinstance(response.json(), list)

# Tests d'authentification
def test_login():
    login_data = {
        "email": "admin@example.com",
        "password": "secret"
    }
    response = client.post("/login", json=login_data)
    assert response.status_code == 200
    assert "access_token" in response.json()

# Tests d'erreurs
def test_get_user_not_found():
    response = client.get("/users/999")
    assert response.status_code == 404

# 👇 Exécutez les tests :
if __name__ == "__main__":
    pytest.main(["-v", "--tb=short"])

In [9]:
# 💡 Aide pour l'étape 7.4.1
api_helper.help("7.4.1")

### 📝 Étape 7.4.2 : Déploiement et monitoring

**Instructions :**
Préparer l'API pour la production.

In [ ]:
# 📝 ÉTAPE 7.4.2 : Configuration production
# Configurez votre API pour la production :

from fastapi import FastAPI
from fastapi.responses import JSONResponse
import logging
from contextlib import asynccontextmanager

# Configuration du logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)
logger = logging.getLogger(__name__)

# Gestionnaire d'événements de cycle de vie
@asynccontextmanager
async def lifespan(app: FastAPI):
    # Startup
    logger.info("API démarrée")
    yield
    # Shutdown
    logger.info("API arrêtée")

# Configuration de production
app_prod = FastAPI(
    title="Production API",
    description="API prête pour la production",
    version="1.0.0",
    lifespan=lifespan,
    docs_url="/docs",  # Swagger UI
    redoc_url="/redoc", # ReDoc
    openapi_url="/openapi.json"
)

# Gestionnaire d'erreurs global
@app_prod.exception_handler(Exception)
async def global_exception_handler(request, exc):
    logger.error(f"Erreur non gérée: {exc}")
    return JSONResponse(
        status_code=500,
        content={"message": "Erreur interne du serveur"}
    )

# Endpoint de santé
@app_prod.get("/health")
async def health_check():
    return {"status": "healthy", "timestamp": datetime.now()}

# 👇 Lancez en mode production :
# uvicorn main:app_prod --host 0.0.0.0 --port 8000 --workers 4

In [10]:
# 💡 Aide pour l'étape 7.4.2
api_helper.help("7.4.2")

---

## 🎊 FÉLICITATIONS !

Vous avez terminé le module API ! Vous maîtrisez maintenant :

### ✅ Ce que vous avez appris :
- **Théorie** : SOAP, REST, GraphQL et architectures d'API
- **FastAPI** : Framework moderne pour créer des APIs Python
- **Validation** : Modèles Pydantic et validation automatique
- **Sécurité** : JWT, OAuth2, CORS et middlewares
- **Tests** : Tests automatisés avec pytest et TestClient
- **Production** : Déploiement, logging et monitoring

### 🚀 Prochaines étapes :
- Explorez GraphQL avec Strawberry ou Ariadne
- Intégrez avec des bases de données (SQLAlchemy, Tortoise ORM)
- Déployez sur le cloud (AWS, GCP, Azure)
- Apprenez les microservices et l'architecture distribuée

**🎯 Votre formation Python Data est maintenant complète !**